In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

In [2]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [3]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(127, 40)
Index(['run_id', 'lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val',
       'train', 'y_low', 'device', 'n_test', 'y_high', 'log_dir', 'n_train',
       'predict', 'version', 'data_dir', 'eps_mult', 'n_splits', 'do_return',
       'noise_std', 'only_flip', 'batch_size', 'max_epochs', 'data_kwargs',
       'num_workers', 'space_length', 'generate_data', 'model_summary',
       'args_processed', 'do_return_model', 'transform_kwargs',
       'persistent_workers', 'run_name', 'test_loss', 'logger', 'grid_size',
       'criterion'],
      dtype='object')


,run_id,lr,net,bias,name,seed,tags,test,n_val,train,...,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,logger,grid_size,criterion
0,2m7eu5tn,0.0010,TrainP,False,None,5.0,['v5'],True,100.0,True,...,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,worldly-snowflake-183,0.000473,NaN,NaN,NaN
1,ugylvzz6,0.0001,TrainP,False,None,5.0,['v5'],True,100.0,True,...,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,classic-glade-184,0.009489,NaN,NaN,NaN
2,unpw27ml,0.1000,TrainP,False,None,5.0,['v5'],True,100.0,True,...,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,sage-river-182,0.003589,NaN,NaN,NaN
3,w4yjcu25,0.0100,TrainP,False,None,5.0,['v5'],True,100.0,True,...,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,azure-grass-181,0.003185,NaN,NaN,NaN
4,q86f7s5e,0.0001,TrainP,False,None,5.0,['v5'],True,100.0,True,...,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,blooming-star-180,0.023643,NaN,NaN,NaN


In [5]:
api = wandb.Api()
filters = {'state': 'finished'}
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}

In [18]:
run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
if not len(run_dirs) == 0:
    assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
    run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
    run_dirs_df = pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir']).sort_values(by = 'run_dir', ascending = False)
else:
    run_dirs_df = pd.DataFrame(columns = ['run_id', 'run_dir'])


for i, values in run_dirs_df.iterrows():
    run_id = values['run_id']
    run_dir = values['run_dir']
    if run_id not in run_id_api:
        print(f'Removing {run_id}, {run_dir}')
        # shutil.rmtree(os.path.join(wandb_dir, values['run_dir']), )

run_dirs_df

Removing yf583iy8, run-20240129_152843-yf583iy8
Removing 2yffuyyh, run-20240125_153218-2yffuyyh
Removing mzjurctg, run-20240125_153211-mzjurctg
Removing kqsdmwsd, run-20240125_153207-kqsdmwsd
Removing uguvwf25, run-20240125_153206-uguvwf25
Removing h78ny35l, run-20240125_153205-h78ny35l
Removing mhzkk0i3, run-20240125_153203-mhzkk0i3
Removing i2klyk2q, run-20240125_153203-i2klyk2q
Removing h5g6wqls, run-20240125_153203-h5g6wqls
Removing 9xu71cw3, run-20240125_153203-9xu71cw3
Removing 6q06sgky, run-20240125_153203-6q06sgky
Removing 1pdj0ehk, run-20240125_153203-1pdj0ehk
Removing colf2qi0, run-20240125_153201-colf2qi0
Removing n103ix5y, run-20240125_153005-n103ix5y
Removing 90hd4qbd, run-20240125_153005-90hd4qbd
Removing v2f6nbkh, run-20240125_153002-v2f6nbkh
Removing n6o8bymo, run-20240125_153002-n6o8bymo
Removing hcurlkrr, run-20240125_153002-hcurlkrr
Removing 3ups9zx0, run-20240125_153002-3ups9zx0
Removing u6j7x5yh, run-20240125_153001-u6j7x5yh
Removing uthwe5o5, run-20240125_152959-u

,run_id,run_dir
68,uegf6m2h,run-20240129_164256-uegf6m2h
154,vz4jfzh6,run-20240129_163734-vz4jfzh6
140,wqmfolq3,run-20240129_163730-wqmfolq3
151,5k72ln34,run-20240129_163725-5k72ln34
54,mg3t7tf9,run-20240129_163226-mg3t7tf9
...,...,...
38,roz9fm0v,run-20240125_135752-roz9fm0v
173,wb53kqmt,run-20240125_135751-wb53kqmt
105,gouerrhp,run-20240125_135751-gouerrhp
192,e60auxdz,run-20240125_135751-e60auxdz


In [38]:
Ps = os.listdir(os.path.join(store_dir, 'P'))
for P in Ps:
    P_id = P.split('.')[0]
    if P_id not in run_id_api:
        print(f'Removing {P_id}')
        # os.remove(os.path.join(store_dir, 'P', P))

NameError: name 'run_id_api' is not defined

In [6]:
exclude_ids = []

In [14]:
tags = 'mnist'

api = wandb.Api()
filters = {'tags': tags} if tags else {}
filters['state'] = 'finished'
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
print(len(runs))
for i, run in enumerate(runs):
    print(run.name, run.id, run.tags)
    # if run.id in exclude_ids: run.delete()

30
bumbling-jazz-403 ytp0ix8i ['hparams', 'long', 'mnist']
graceful-puddle-401 fmxhxywt ['hparams', 'long', 'mnist']
serene-cloud-402 qd8q06l7 ['hparams', 'long', 'mnist']
dark-deluge-398 85tbv8pi ['hparams', 'long', 'mnist']
absurd-surf-400 evjpekkf ['hparams', 'long', 'mnist']
clear-vortex-399 p7hwjop0 ['hparams', 'long', 'mnist']
hopeful-armadillo-397 ztv38avp ['hparams', 'mnist']
sunny-serenity-368 faww6jy0 ['batchsize_check2', 'hparams', 'mnist']
summer-dragon-367 fw1y9anh ['batchsize_check2', 'hparams', 'mnist']
devout-meadow-365 0swho700 ['batchsize_check2', 'hparams', 'mnist']
different-flower-366 f4mckbyv ['batchsize_check2', 'hparams', 'mnist']
giddy-terrain-364 ld1nxjev ['batchsize_check2', 'hparams', 'mnist']
dulcet-mountain-362 9q4b9za1 ['batchsize_check', 'hparams', 'mnist']
legendary-grass-334 nda993x3 ['hparams', 'mnist']
smooth-pine-333 ye20nvmv ['hparams', 'mnist']
crisp-serenity-332 0u51wq8q ['hparams', 'mnist']
eager-plant-330 n3n0r29a ['hparams', 'mnist']
helpful-s

In [6]:
listdir_dirs = lambda path: [entry for entry in os.listdir(path) if os.path.isdir(os.path.join(path, entry))]
store_dir_dirs = listdir_dirs(store_dir)
store_dir_dirs

['P']

In [15]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue
    if 'dev' in run.tags:
        continue


    # print(id)
    # continue

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()

    if 'test_loss' not in history:
        continue


    metrics = ['test_loss'] # 'train_loss', 'val_loss'
    for metric in metrics:

        config[metric] = np.array(history[metric].dropna()).item()

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb n0oowy43: 100%|██████████| 30/30 [00:05<00:00,  5.81it/s]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,data_kwargs,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss
0,0.001,TrainP,False,None,3,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,bumbling-jazz-403,0.004913
1,0.010,TrainP,False,None,3,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,graceful-puddle-401,0.001697
2,0.010,TrainP,False,None,2,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,serene-cloud-402,0.002914
3,0.001,TrainP,False,None,2,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,dark-deluge-398,0.005096
4,0.010,TrainP,False,None,1,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,absurd-surf-400,0.002773
5,0.001,TrainP,False,None,1,"[mnist, hparams, long]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,clear-vortex-399,0.004782
6,0.010,TrainP,False,None,42,"[mnist, hparams]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0}",7,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,hopeful-armadillo-397,0.016291
7,0.001,TrainP,False,None,42,"[mnist, hparams, batchsize_check2]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0.5}",18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,sunny-serenity-368,0.036017
8,0.001,TrainP,False,None,42,"[mnist, hparams, batchsize_check2]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0.5}",18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,devout-meadow-365,0.038581
9,0.001,TrainP,False,None,42,"[mnist, hparams, batchsize_check2]",True,1000,True,None,...,"{'grid_size': [7, 7], 'noise_std': 0.5}",18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,different-flower-366,0.022030


In [16]:
results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))

/var/folders/_t/q2qhsxb97lj90dx67tklgrq40000gn/T/ipykernel_36791/4242539017.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
